##1. Create connect to blob storage for reading data ##

In [0]:
service_credential = dbutils.secrets.get(scope="is402cloud-keyvault ",key="cloud-secret")

spark.conf.set("fs.azure.account.auth.type.is402storage.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.is402storage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.is402storage.dfs.core.windows.net", "b1959452-d9a0-4f1e-bf3b-f4fd858011e1")
spark.conf.set("fs.azure.account.oauth2.client.secret.is402storage.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.is402storage.dfs.core.windows.net", "https://login.microsoftonline.com/328cb62a-06ea-4cfb-b7bf-816691f80770/oauth2/token")

In [0]:
df1 = spark.read.format("csv").option("header", "True").option("inferSchema", "True").load("abfss://output@is402storage.dfs.core.windows.net/taxi_partition_1.csv")
df2 = spark.read.format("csv").option("header", "True").option("inferSchema", "True").load("abfss://output@is402storage.dfs.core.windows.net/taxi_partition_2.csv")

##2. Create and Use Database "taxi_database" ##

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS taxi_database;
USE taxi_database;

##3. Create Bronze Table for Raw Data ##

In [0]:
%sql
CREATE TABLE IF NOT EXISTS taxi_bronze(
  VENDORID INT,
  TPEP_PICKUP_DATETIME TIMESTAMP,
  TPEP_DROPOFF_DATETIME TIMESTAMP,
  TRIP_DISTANCE FLOAT,
  TOTAL_AMOUNT FLOAT
)

In [0]:
# Cast column trip_distacne type to float
df1 = df1.withColumn("trip_distance", df1.trip_distance.cast('float'))
df2 = df2.withColumn("trip_distance", df2.trip_distance.cast('float'))

# Cast column total_amount type to float
df1 = df1.withColumn("total_amount", df1.total_amount.cast('float'))
df2 = df2.withColumn("total_amount", df2.total_amount.cast('float'))

In [0]:
# Write into table taxi_brone by df1
df1.write.insertInto("taxi_bronze")

# Write into table taxi_brone by df2
df2.write.insertInto("taxi_bronze")

In [0]:
# Order data by "total_amount"
df_taxi_silver = spark.sql('''SELECT * 
                        FROM taxi_bronze
                        ORDER BY total_amount ASC''')

##4. Create Silver Table for Processed Data ##

In [0]:
%sql
CREATE TABLE IF NOT EXISTS taxi_silver(
  VENDORID INT,
  TPEP_PICKUP_DATETIME TIMESTAMP,
  TPEP_DROPOFF_DATETIME TIMESTAMP,
  TRIP_DISTANCE FLOAT,
  TOTAL_AMOUNT FLOAT
)

In [0]:
df_taxi_silver.write.insertInto("taxi_silver")